In [1]:
import pandas as pd
import numpy as np
import os
import random

In [2]:
# load imaging data

# load CSA data
area_items = pd.read_csv('/data/sliu/imaging_genetics/DK_structure/Area_items.csv')
s1 = ['eid']
for i in range(area_items.shape[0]):
    s1.append(str(area_items.iloc[i,0])+'-2.0')
data1 = pd.read_csv('/data/sliu/UKBB_phenotypes/ukb42608.csv',usecols=s1)
data1.dropna(axis=0,how='any',inplace=True)
# load CT data
thickness_items = pd.read_csv('/data/sliu/imaging_genetics/DK_structure/Thickness_items.csv')
s2= ['eid']
for i in range(thickness_items.shape[0]):
    s2.append(str(thickness_items.iloc[i,0])+'-2.0')
data2 = pd.read_csv('/data/sliu/UKBB_phenotypes/ukb42608.csv',usecols=s2)
data2.dropna(axis=0,how='any',inplace=True)

#load FC data
temp_FC_data = pd.read_csv('/data/sliu/subsampling_analysis/ukbb_FC.csv')

In [3]:
# select rsfMRI head motion, total gray matter volume, ICV, total cortical surface area, average cortical thickness, scanning sites
cols = ['eid','25741-2.0','25005-2.0','26521-2.0','26721-2.0','26822-2.0','26755-2.0','26856-2.0','54-2.0']

cov1 = pd.read_csv('/data/sliu/UKBB_phenotypes/ukb42608.csv',usecols=cols)
cov1.dropna(how='any',inplace=True)

sites1 = []
sites2 = []
sites3 = []
for i in range(cov1.shape[0]):
    if cov1['54-2.0'].iloc[i] == 11025:
        sites1.append(1)
        sites2.append(0)
        sites3.append(0)
    if cov1['54-2.0'].iloc[i] == 11026:
        sites1.append(0)
        sites2.append(1)
        sites3.append(0)
    if cov1['54-2.0'].iloc[i] == 11027:
        sites1.append(0)
        sites2.append(0)
        sites3.append(1)
cov1['site1'] = sites1
cov1['site2'] = sites2
cov1['site3'] = sites3

In [4]:
# select sex, age, variable
# numeric memory: 4282-2.0 (this example)
# fluid intelligence: 20016-2.0
# Body mass index: 21001-2.0
# birth month: 52-0.0
# Age: 21003-2.0

cols2 = ['eid','31-0.0','21003-2.0','4282-2.0']
cov2 = pd.read_csv('/data/sliu/UKBB_phenotypes/ukb30545.csv',usecols=cols2)
cov2.dropna(how='any',inplace=True)
cov2 = cov2[cov2['4282-2.0'] > 0]
cov22 = cov2.set_index('eid')

# if alcohol consumption
# Alchol = pd.read_csv('/data/sliu/new_gene_expression_analysis/AlcFRQ.pheno',sep=' ')
# Alchol.rename(columns={'FID':'eid'},inplace=True)
# cov2 = cov2.merge(Alchol, how='inner', on='eid')

#if neuroticism
# select sex, age, IQ, EA from Karin's ukbb data and matched IDs with Guido's ukbb data
# cols2 = ['eid','31-0.0','21003-2.0','1920-2.0', '1930-2.0', '1940-2.0', '1950-2.0',\
#          '1960-2.0', '1970-2.0', '1980-2.0', '1990-2.0', '2000-2.0', '2010-2.0', '2020-2.0', '2030-2.0']
# cov2 = pd.read_csv('/data/sliu/UKBB_phenotypes/ukb30545.csv',usecols=cols2)
# cov2.dropna(how='any',inplace=True)

# cov2['Neuroticism'] = np.sum(cov2.iloc[:,2:-1].values == 1, axis=1)
# cols22 = ['eid','31-0.0','21003-2.0','Neuroticism']
# cov22 = cov2[cols22].set_index('eid')

In [5]:
# merge data
temp_FC_data.drop_duplicates(['eid'],inplace=True)
temp_FC_data2 = temp_FC_data.set_index('eid')
temp_FC_data2.dropna(how='any',inplace=True)

temp_l = list(set(cov22.index) & set(temp_FC_data2.index))
random.shuffle(temp_l)
matched_IDs = pd.read_csv('/data/sliu/updated_ukbb2/matched_IDs2.csv')
matched_IDs.set_index('karin_id',inplace=True)
temp_l2 = list(set(temp_l) & set(matched_IDs.index))
guido_ids = matched_IDs.loc[temp_l2]['guido_id'].values

temp_FC_data3 = temp_FC_data2.loc[temp_l2]
temp_FC_data3.reset_index(inplace=True)
temp_FC_data3['guido_id'] = guido_ids
temp_FC_data3.dropna(how='any',inplace=True)
temp_FC_data3.rename(columns={'eid':'karin_id','guido_id':'eid'},inplace=True)

info_pheno = cov22.loc[temp_l2]
info_pheno.reset_index(inplace=True)
info_pheno['guido_id'] = guido_ids
info_pheno.dropna(how='any',inplace=True)
info_pheno.rename(columns={'eid':'karin_id','guido_id':'eid'},inplace=True)

data11 = data1.set_index('eid')
data22 = data2.set_index('eid')
cov11 = cov1.set_index('eid')
temp_FC_data4 = temp_FC_data3.set_index('eid')
info_pheno2 = info_pheno.set_index('eid')

l = list(set(data11.index) & set(data22.index) & set(cov11.index) & set(info_pheno2.index) & set(temp_FC_data4.index))
CSA_data = data11.loc[l]
CT_data = data22.loc[l]
FC_data = temp_FC_data4.loc[l]
FC_data.drop(['karin_id'],axis=1,inplace=True)
T_data = cov11.loc[l]
info_data = info_pheno2.loc[l]

# total data including the variable and covariates
df = pd.DataFrame({"ID" : info_data.index})
df['Age'] = info_data['21003-2.0'].values
df['Sex'] = info_data['31-0.0'].values
df['variable'] = info_data['4282-2.0'].values
df['motion'] = T_data['25741-2.0'].values
df['TGMV'] = T_data['25005-2.0'].values
df['ICV'] = T_data['26521-2.0'].values
df['TCSA'] = T_data['26721-2.0'].values + T_data['26822-2.0'].values
df['ACT'] = T_data['26755-2.0'].values + T_data['26856-2.0'].values
df['site1'] = T_data['site1'].values
df['site2'] = T_data['site2'].values
df['site3'] = T_data['site3'].values

In [6]:
# covariates
co1_items = ['Age','Sex','TCSA','site1','site2','site3']
co1 = df[co1_items].values
co2_items = ['Age','Sex','ACT','site1','site2','site3']
co2 = df[co2_items].values
co3_items = ['Age','Sex','motion','site1','site2','site3']
co3 = df[co3_items].values
co4_items = ['Age','Sex','site1','site2','site3']
co4 = df[co4_items].values

In [7]:
from sklearn.preprocessing import StandardScaler

def regression_covariant(covariant_matrix, y, standard_scale=False):
    a = np.hstack((covariant_matrix,np.ones((covariant_matrix.shape[0], 1))))
    w = np.linalg.lstsq(a,y,rcond=None)[0]

    residual = y - covariant_matrix.dot(w[:-1])
    residual = residual.astype('float64')

    if standard_scale:
        residual = StandardScaler().fit_transform(residual.reshape(-1,1)).flatten()

    return residual, w

In [8]:
def regress_data(data,co):
    s1 = data.shape
    reg_data = np.zeros(s1)
    for i in range(s1[1]):
        x = data.iloc[:,i].values
        [rx,w1] = regression_covariant(co,x,standard_scale=False)
        reg_data[:,i] = rx
    return reg_data

In [9]:
#output the data which has already regressed out the covariates
output_path = 'data/Numeric_memory/'
reg_CSA_data = regress_data(CSA_data,co1)
reg_CT_data = regress_data(CT_data,co2)

my_CSA = pd.DataFrame(data=reg_CSA_data)
my_CSA['eid'] = l
my_CSA.to_csv(output_path+'CSA_with_controlling_for_total_brain.csv',index=False)

my_CT = pd.DataFrame(data=reg_CT_data)
my_CT['eid'] = l
my_CT.to_csv(output_path+'CT_with_controlling_for_total_brain.csv',index=False)

reg_FC_data = regress_data(FC_data,co3)
my_FC = pd.DataFrame(data=reg_FC_data)
my_FC['eid'] = l
my_FC.to_csv(output_path+'FC_data.csv',index=False)

co5_items = ['Age','Sex','TCSA','ACT','motion','site1','site2','site3']
co5 = df[co5_items].values
variable = df['variable'].values

rvariable,w = regression_covariant(co5,variable,standard_scale=False)
my_variable = pd.DataFrame(data=rvariable)
my_variable['eid'] = l
my_variable.to_csv(output_path+'variable_with_controlling_for_total_brain.csv',index=False)